<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 4 / SPRINT 1 / NOTE 2*

---

# Vectorization of Texts

* 텍스트 문서를 벡터로 표현해 봅시다
* 유사도를 이용해 문서를 검색해 봅시다
* 문서를 벡터로 만들기 위해 단어 임베딩을 사용해 봅시다

### Warm up


#### Vectorization of Texts(텍스트 벡터화)

컴퓨터는 자연어를 있는 그대로 이해할 수 없습니다. 그래서 자연어를 컴퓨터가 사용할 수 있는 형태로 가공해야 합니다. 이전 강의에서는 정제된 토큰의 수를 기반으로 분석을 진행하였습니다. 오늘은 이 개념을 좀 더 확장시켜 **Bag-of-Words(BoW), TF-IDF** 같은 텍스트를 벡터화 하는 방법과 **단어 임베딩(Word embedding) 모델**에 대해 다루어 보겠습니다. 다음 수업에 진행할 검색, 시각화, 분류 작업에 벡터 표현법을 활용할 것입니다.

머신러닝 모델에서 사용하기 위해 텍스트 데이터를 벡터화하는 것은 컴퓨터가 사용할 수 있는 수치정보로 변환하는것으로 생각할 수 있습니다.

Bag-of-Words(BoW) 모델은 문장이나 문서들에서 문법, 단어 순서 등의 개념을 제거하여 단순히 **단어들의 빈도**만 고려하는 모델입니다.

BoW는 문서를 토큰화한 후 토큰의 빈도를 기반으로 벡터화 합니다. 데이터프레임 형태로 보자면 행은 각 문서가 되고 열은 중복되지 않는 각 단어가 됩니다. 열에는 단순히 각 단어가 문서에 얼마나 존재하는지를 카운트한 값을 넣거나(*CountVectorizer* 사용) TF-IDF 값이 오게 할 수 있습니다(*TfidfVectorizer 사용).

벡터 표현은 파이썬에서는 `sklearn`, `spacy`패키지를 사용해 구현할 수 있습니다.

#### 다음 영상을 시청하세요
- [TF-IDF](https://youtu.be/meEchvkdB1U)
- [Nearest Neighbor Learning | Stanford University](https://youtu.be/ONM5MB3_iOU)


#### Dataset

이 모듈에서는 BBC에서 제공하는 데이터셋을 사용합니다. BBC 웹사이트를 방문하는 고객이 방금 읽은 문서를 기반으로 비슷한 다른 문서를 적절하게 추천할 수 있을까요?

다음 파일을 다운로드 받아 노트 폴더에서 압축을 해제하세요. data folder가 생성되고 001.txt~401.txt 파일이 있는지 확인합니다.

[bbc_fulltext.zip](https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/bbc_fulltext/bbc_fulltext.zip)


#### 이 데이터의 소스는 다음과 같습니다.
[BBC News Tech]('https://www.bbc.com/news/technology')

[D. Greene and P. Cunningham. "Practical Solutions to the Problem of Diagonal Dominance in Kernel Document Clustering", Proc. ICML 2006.](http://mlg.ucd.ie/datasets/bbc.html)
**(tech 관련 문서만 사용합니다)**
파일을 직접 받으려면 위 링크에서, Dataset: BBC -> >> Download raw text files -> 압축해제(bbc-fulltext.zip)> tech 폴더 내 txt 파일(001.txt ~ 401.txt)



---

## 텍스트 문서를 벡터로 표현해 봅시다

BoW를 사용해 Document Term Matrices(DTM, 문서-단어행렬)을 만들어 보겠습니다. 각 행은 문서를 나타내고 각 열은 단어를 나타냅니다.
- 각 셀의 값은 여러가지 방법으로 표현될 수 있는데
    - 단어의 출현 빈도를 나타내거나,
    - 단순히 단어의 존재 유무(binary)를 표현할 수 있고,
    - term-frequency inverse-document (TF-IDF) 값으로 나타낼 수 있습니다.

In [ ]:
# 모듈에서 사용할 라이브러리와 spacy 모델을 불러옵니다.

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

import spacy
nlp = spacy.load("en_core_web_lg")

**Spacy로 텍스트에서 토큰을 추출해 보겠습니다.**

In [ ]:
text = "In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. It is often used as a weighting factor in searches of information retrieval, text mining, and user modeling. The tf–idf value increases proportionally to the number of times a word appears in the document and is offset by the number of documents in the corpus that contain the word, which helps to adjust for the fact that some words appear more frequently in general. tf–idf is one of the most popular term-weighting schemes today. A survey conducted in 2015 showed that 83% of text-based recommender systems in digital libraries use tf–idf."

In [ ]:
doc = nlp(text)

print([token.lemma_ for token in doc if (token.is_stop != True) and (token.is_punct != True)])

In [ ]:
import os 

# BBC 데이터를 불러오기 위한 함수입니다.

def gather_data(filefolder):
    """ 폴더 내 텍스트 파일을 각각 리스트 요소에 저장하는 함수
    Args:
        filefolder (str): .txt 파일이 존재하는 경로
    Returns:
        문서를 요소로하는 리스트
    """
    
    data = []
    
    files = os.listdir(filefolder)
    
    for article in files: 
        path = os.path.join(filefolder, article)

        # txt로 끝나는 파일만 읽습니다
        if  path[-3:] == 'txt':
            # rb:Read the file in Binary mode
            with open(path, 'rb') as f:
                data.append(f.read())
    
    return data

bbc tech 관련 문서들을 불러옵니다.

In [ ]:
data = gather_data('./data')

In [ ]:
data[0]

In [ ]:
import seaborn as sns
# plot 스타일과 폰트 크기를 설정합니다.
sns.set(style='whitegrid', font_scale=1.15)

# 문서별 단어의 수 분포도 그리는 함수
def plot_text_length_dist(text_list):

    # 문장이 요소인 리스트를 받아 각 문서의 단어 수를 가진 리스트를 만듭니다
    num_words = [len(doc.split()) for doc in text_list]
    
    sns.displot(num_words)
    plt.title('# of words per documents')
    plt.xlabel('Number of words')
    plt.ylabel('Number of documents')
    plt.show()       

대략 500 단어 정도로 표현된 문서가 가장 많이 보입니다.

In [ ]:
plot_text_length_dist(data)

### CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# wiki 문장들을 리스트 내에 나누어 저장하였습니다
text = ["In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus."
,"It is often used as a weighting factor in searches of information retrieval, text mining, and user modeling."
,"The tf–idf value increases proportionally to the number of times a word appears in the document and is offset by the number of documents in the corpus that contain the word, which helps to adjust for the fact that some words appear more frequently in general."
,"tf–idf is one of the most popular term-weighting schemes today."
,"A survey conducted in 2015 showed that 83% of text-based recommender systems in digital libraries use tf–idf."]

# CountVectorizer 생성
vect = CountVectorizer()

# text를 기반으로 어휘 사전을 생성
vect.fit(text)

# text를 DTM(document-term matrix)으로 변환(transform)
dtm = vect.transform(text)

vocabulary(모든 토큰)와 맵핑된 인덱스 정보를 확인할 수 있습니다

In [ ]:
vect.vocabulary_

In [ ]:
dtm.shape

추출된 토큰을 나열해 봅니다.

In [ ]:
print(vect.get_feature_names())

In [ ]:
text[0]

dtm의 타입을 보면 compressed sparse Row matrix임을 알 수 있습니다. csr: Compressed Sparse Row matrix, sparse matrix 형태에서 0을 표현하지 않습니다.

In [ ]:
type(dtm)

In [ ]:
# (row, column)  count
print(dtm)

0을 표현한 형태로 만들면 다음과 같이 됩니다.

In [ ]:
# Return a dense matrix representation
dtm.todense()

데이터프레임 형태로 결과를 보고 싶다면

In [ ]:
dtm = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names())
dtm

세번째 문장과, dtm을 비교해 보겠습니다. 

In [ ]:
text[2]

**CountVectorizer를 BBC data에 적용해 봅시다**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

## stop_words = 'english' 영어에 해당하는 불용어 처리를 합니다.
## max_features=n, 빈도 순서대로 top n 단어만 사용합니다.
vect = CountVectorizer(stop_words='english'
                       , max_features=10000)

# fit & transform
dtm = vect.fit_transform(data)

dtm = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names())
dtm.shape

In [ ]:
dtm.head()

### TfidfVectorizer

#### Term Frequency - Inverse Document Frequency (TF-IDF, 단어빈도-역문서빈도)
- [TF-IDF 참고]("https://mungingdata.wordpress.com/2017/11/25/episode-1-using-tf-idf-to-identify-the-signal-from-the-noise/")

*t=단어, d=문서, n=총 문서수*

$TF-IDF\; Score = \large tf(t,d) \times idf(t)$
<br></br>

TF(Term Frequency, 단어의 빈도), **특정 문서 d에서 특정 단어 t가 쓰인 빈도**:

$\large tf(t,d) = \frac{Term\; t\; frequency\; in\; document}{Total\; words\; in\; document}$
<br></br>

DF(Document Frequency, 문서의 빈도), **특정 단어 t가 나타난 문서의 갯수**:

$\large df(t) = \frac{documents\; with\; term\; t}{Total\; documents}$
<br></br>

IDF : log(DF 역수(inverse))

$\large idf(t) = log(\frac{n}{1+df(t)})$
<br></br>

TF-IDF 를 사용하는 이유는 문서를 구분하는데 어떤 단어가 중요한지(**unique**) 찾는 것 입니다.

수식을 살펴보면, 여러 문서에서 많이 등장하는 단어일 수록 중요도가 낮다고 판단하며(IDF), 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단합니다(TF).

TF-IDF를 통한 자연어 처리는 간단하고 빠르게 구현할 수 있으므로 좋은 Baseline으로 사용할 수 있습니다.

#### Tfidf vectorizer vs Count vectorizer
TF-IDF vectorizer를 생성하고 dtm을 만들어 보겠습니다.

In [ ]:
# TF-IDF vectorizer. 테이블을 작게 만들기 위해 max_features=15로 제한하였습니다.
tfidf = TfidfVectorizer(stop_words='english', max_features=15)

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm = tfidf.fit_transform(text)

dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
dtm

같은 파라미터로 CountVectorizer를 사용해 tfidf 결과와 비교해 보겠습니다.

In [ ]:
vect = CountVectorizer(stop_words='english', max_features=15)
dtm = vect.fit_transform(text)
dtm = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names())
dtm

#### BBC 데이터에 tfidf vectorizer를 적용해 보겠습니다.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
dtm = tfidf.fit_transform(data)
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

dtm.head()

#### 이번에는 조금 더 파라미터를 튜닝하고, spacy tokenizer 를 사용해서 벡터화를 진행해 보겠습니다.

In [ ]:
# spacy tokenizer 함수
def tokenize(document):
    
    doc = nlp(document)
    # punctuations: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

파라미터 튜닝을 더 해보겠습니다. 여러 파라미터들을 변경해 가며 결과를 비교해 보겠습니다.

In [ ]:
# ngram_range = (min_n, max_n), min_n 개~ max_n 개를 갖는 n-gram(n개의 연속적인 토큰)을 토큰으로 사용합니다.
# min_df = n, 최소 n개의 문서에 나타나는 토큰만 사용합니다
# max_df = .7, 70% 이상 문서에 나타나는 토큰은 제거합니다
tfidf = TfidfVectorizer(stop_words='english'
#                         ,tokenizer=tokenize
                        ,ngram_range=(1,2)
                        ,max_df=.7
                        ,min_df=3
#                         ,max_features = 4000
                       )

dtm = tfidf.fit_transform(data)
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
dtm.head()

In [ ]:
dtm.shape

## 유사도를 이용해 문서를 검색해 봅시다

검색엔진의 원리를 생각해 보셨나요? 검색어를 인터넷에 존재하는 여러 문서들과 단순히 같은지만 비교 하는것은 아닙니다. 쿼리와 문서들을 매칭(matching) 하는 방법은 여러가지가 있습니다. 그중 가장 클래식한 방법으로 유사도를 측정하기 위해 n-차원 거리를 사용하는 방법을 살펴보겠습니다.

### 코사인 유사도(Cosine Similarity, Brute Force 방법)

$\Large similarity=cos(Θ)=\frac{A⋅B}{||A||\ ||B||}=\frac{\sum_{i=1}^{n}{A_{i}×B_{i}}}{\sqrt{\sum_{i=1}^{n}(A_{i})^2}×\sqrt{\sum_{i=1}^{n}(B_{i})^2}}$

<img align="center" src="https://images.deepai.org/glossary-terms/cosine-similarity-1007790.jpg" width=700 title="Cosine Similarity" alt="https://deepai.org/machine-learning-glossary-and-terms/cosine-similarity">


코사인 유사도는 두 벡터(문서벡터) 간의 각의 코사인 값을 이용하여 구할 수 있는 유사도 입니다.
- 두 벡터(문서)가 
    - 완전히 같을 경우 1이며
    - 90도의 각을 이루면 0
    - 완전히 반대방향을 이루면 -1 입니다

In [ ]:
dtm.shape

#### TF-IDF 벡터들의 거리를 계산해 보겠습니다
[sklearn.metrics.pairwise.cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html#sklearn-metrics-pairwise-cosine-similarity)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# input, X:(n_samples_X, n_features)
distance_matrix  = cosine_similarity(dtm)

In [ ]:
df = pd.DataFrame(distance_matrix)

유사도는 문서 x 문서 행렬로 표현됩니다

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
data[0][:100]

#### 문서0과 문서0은 같으므로 유사도가 1입니다. 문서0과 문서1~4 와의의 유사도를 확인해 보세요.

In [ ]:
df[0][:5]

#### 문서0과 유사도가 큰 문서를 순서대로 정렬해서 살펴보겠습니다 (TF-IDF vectorization 방법에 따라 많이 다를 수 있습니다)

In [ ]:
df[df[0] < 1][0].sort_values(ascending=False)[:5]

In [ ]:
print(data[0][:100])
print(data[92][:100])

코사인 유사도와 같은 Brute Force 방법은 비교해야 할 문서의 양이 많아 질 수록 많은 계산을 필요로합니다. 배포 환경에서는 더 빠른 비교 방법을 사용해야 합니다.

### NearestNeighbor (K-NN, K-최근접 이웃) 

K-최근접 이웃법은 쿼리와 가장 가까운 상위 K개의 근접한 데이터를 찾아서 K개 데이터의 유사성을 기반으로 점을 추정하거나 분류하는 예측 분석에 사용됩니다.
최근접 이웃 방법은 non-generalizing 머신러닝 방법인데, 모든 학습 데이터를 KD Tree 나 Ball Tree같은 빠른 색인 구조(indexing structure)에 단순히 저장하기 때문입니다.

K-D 트리의 기본아이디어는 점 A와 B가 멀고, B가 C와 가까우면 A가 C와 아주 멀다는 것을 알 수 있는데 이때 명시적으로 A와 C의 거리를 계산할 필요가 없다는 것입니다.

[K-D Tree]('https://scikit-learn.org/stable/modules/neighbors.html?highlight=very%20distant%20from%20point#ball-tree')
> To address the computational inefficiencies of the brute-force approach, a variety of tree-based data structures have been invented. In general, these structures attempt to reduce the required number of distance calculations by efficiently encoding aggregate distance information for the sample. The basic idea is that if point *A* is very distant from point *B*, and point *B* is very close to point *C*, then we know that points *A* and *C* are very distant, without having to explicitly calculate their distance. In this way, the computational cost of a nearest neighbors search can be reduced to *O(DNlog(N))* or better. This is a significant improvement over brute-force for large *N*.

Ball Tree는 K-D 트리를 더욱 효율적으로 만들기 위해 개발되었습니다. KD 트리는 데이터를 Cartesian 축으로 분할하지만 Ball Tree는 nesting hyper-spheres 형태로 분할하여 트리구성에 비용이 더 들지만, 매우 구조화된 데이터나 높은 차원의 데이터에 더 효율적입니다.

[Ball Tree]('https://scikit-learn.org/stable/modules/neighbors.html?highlight=very%20distant%20from%20point#ball-tree')
> To address the inefficiencies of KD Trees in higher dimensions, the ball tree data structure was developed. Where KD trees partition data along Cartesian axes, ball trees partition data in a series of nesting hyper-spheres. This makes tree construction more costly than that of the KD tree, but results in a data structure which can be very efficient on highly structured data, even in very high dimensions.

> A ball tree recursively divides the data into nodes defined by a centroid *C* and radius *r*, such that each point in the node lies within the hyper-sphere defined by *r* and *C*. The number of candidate points for a neighbor search is reduced through use of the triangle inequality:
$$|x+y| \leq |x| + |y|$$

> With this setup, a single distance calculation between a test point and the centroid is sufficient to determine a lower and upper bound on the distance to all points within the node. Because of the spherical geometry of the ball tree nodes, it can out-perform a KD-tree in high dimensions, though the actual performance is highly dependent on the structure of the training data. In scikit-learn, ball-tree-based neighbors searches are specified using the keyword `algorithm = 'ball_tree'`, and are computed using the class `sklearn.neighbors.BallTree`. Alternatively, the user can work with the `BallTree` class directly.

In [ ]:
dtm.head()

#### sklearn에서 비지도학습을 위한 NearestNeighbors 모델을 사용합니다
[sklearn.neighbors.NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn-neighbors-nearestneighbors)

In [ ]:
from sklearn.neighbors import NearestNeighbors

# dtm을 사용히 NN 모델을 학습시킵니다. (디폴트)최근접 5 이웃.
nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm)

#### 문서0과 가장 가까운 문서 (0 포함) 5개의 거리(값이 작을수록 유사합니다)와, 문서의 인덱스를 알 수 있습니다

In [ ]:
nn.kneighbors([dtm.iloc[0].values])

#### 문서0의 이웃인 문서297로 검색해 보겠습니다 

In [ ]:
nn.kneighbors([dtm.iloc[297]])

In [ ]:
print(data[297][:300])
print(data[92][:300])

In [ ]:
# https://edition.cnn.com/2020/07/30/tech/huawei-samsung-q2-hnk-intl/index.html
cnn_tech_article = [ """
Hong Kong (CNN Business)Huawei became the world's top smartphone seller last quarter, overtaking Samsung for the first time ever, according to an independent market research report released Thursday.
The Chinese tech company shipped 55.8 million phones in the three months ended in June, surpassing longtime rival Samsung, which shipped 53.7 million, according to the Canalys report.
"Taking first place is very important for Huawei," said Canalys analyst Mo Jia. "It is desperate to showcase its brand strength to domestic consumers, component suppliers and developers."
A years-long US pressure campaign against Huawei has handicapped the Shenzhen-based firm's global business.
Huawei still suffered an annual decline in smartphone shipments of 5%. But Samsung's was a lot bigger at 30%, according to Canalys.
The market research firm said Huawei's victory over Samsung wouldn't have happened without Covid-19. The company was able to take advantage of the economic recovery in China, where Huawei now sells over 70% of its smartphones. Samsung has a very small presence in China.
Huawei&#39;s hopes of global domination have been dashed
Huawei's hopes of global domination have been dashed
Huawei's global smartphone and telecom gear business continues to suffer the fallout from US sanctions that cut the company off from key American tech and supplies.
Without access to popular Google (GOOGL GOOGLE) apps such as YouTube, maps and Gmail, Huawei's latest smartphones are a lot less attractive to international buyers. That will make it very difficult for Huawei to hold on to the global No. 1 position, according to Jia.
"It will be hard for Huawei to maintain its lead in the long term. Its major channel partners in key regions, such as Europe, are increasingly wary of ranging Huawei devices, taking on fewer models, and bringing in new brands to reduce risk. Strength in China alone will not be enough to sustain Huawei at the top once the global economy starts to recover," he said.
"Our business has demonstrated exceptional resilience in these difficult times," Huawei spokeswoman Evita Cao said. Cao did not respond to questions on how the company can maintain its lead going forward.
Huawei's victory came on the same day Samsung posted a big profit bump for the second quarter, with strong chip demand helping the company weather the fallout from the coronavirus pandemic.
Samsung reported operating profit of 8.15 trillion won ($6.8 billion) for the three months that ended in June, up more than 23% compared to the same period last year.
Samsung said sales fell about 6% to 53 trillion won ($44.6 billion).
Shares in Samsung were last up 0.7% in Seoul. South Korea's Kospi (KOSPI) rose 0.1%.
Taiwan&#39;s TSMC is becoming one of the world&#39;s top companies. Intel&#39;s problems are helping
Taiwan's TSMC is becoming one of the world's top companies. Intel's problems are helping
Despite the double digit declines in annual smartphone shipments for the quarter noted by the Canalys report, Samsung reported that the unit remained profitable thanks to savings on marketing costs. (Samsung does not break out specifics about its smartphone shipments, but noted that they declined.)
For the second half of 2020, however, Samsung is warning that "uncertainties related to Covid-19 linger" for its mobile business.
That could be enough to drag the company to revenue losses for the year, according to research firm Crisp Idea.
The consumer electronics unit, which includes smartphones and TVs, is "expected to decline significantly as Covid-19 affects demand and leads to store and plant closures globally," Crisp Idea analysts wrote in a note earlier this month.
Smartphone shipments worldwide are expected to fall about 18% in the first half of the year as the pandemic continues to affect consumer spending, analysts at IDC said in a note last month.
The market research firm added that global smartphone shipments are not expected to return to growth until the first quarter of 2021.
That would also hurt Samsung's memory chip business, because the company supplies chips for rival smartphone companies such as Apple (AAPL) and Huawei."""]


무작위로 선택한 BBc Tech 뉴스를 쿼리로 쓰기 위해 학습된 tfidf vectorizer를 통해 변환하겠습니다

In [ ]:
new = tfidf.transform(cnn_tech_article)
new

In [ ]:
nn.kneighbors(new.todense())

In [ ]:
# 가장 가깝게 나온 문서를 확인합니다 
data[250]

## 문서를 벡터로 만들기 위해 단어 임베딩을 사용해 봅시다

### BoW와 달리 Word2Vec과 같은 단어 임베딩 방법은 문맥(context)정보를 보존합니다

BoW는 단어의 존재 여부와 그 빈도 정보를 중요하게 다루는 대신 단어의 순서 정보를 무시하여 단어 주변 문맥정보를 잃어버린다는 단점이 있습니다.

이와 달리 단어 임베딩 방법중 하나인 Word2Vec은 같이 사용되는 단어 정보를 중요시 하여 벡터화할 때 문맥 정보를 보존합니다. 그래서 의미적 또는 구조적으로 비슷한 사용법을 가진 단어들을 알 수 있게 됩니다.

#### *임베딩이란?*
자연어를 컴퓨터가 이해할 수 있는 수의 나열인 벡터 형태로 바꾸는 과정 또는 결과를 의미합니다. 앞서 살펴본 BoW 방법들은 문서를 벡터화 한 것이라 볼 수 있습니다. 

### Word2Vec 이란?
Word2Vec은 구글 연구팀이 발표한(Mikolov et al., 2013) 기법으로 가장 널리 쓰이는 단어 임베딩 모델 중 한 가지 입니다. 단어 임베딩 방법으로는 skip-gram과 CBOW 두 모델이 제안되었습니다. 

#### 분포가설(Distributional Hypothesis)

Word2Vec이 어떻게 문맥 정보를 보존하는지 이해하려면 분포가설([Distributional Hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics))을 알아야 합니다. 
분포가설은 비슷한 문맥에서 등장하는 단어들은 비슷한 의미를 지닌다는 것 입니다. 여기서 분포(distribution)란 특정 윈도우(window) 범위 안에 동시에 등장하는 이웃 단어나 문맥의 집합을 말합니다. 

예를 들어 두 문장

- I found **good** stores.
- I found **bad** stores.

에서 **good**과 **bad**은 주변단어들이 매우 유사함으로 추축하건데 비슷한 의미를 지닐 것이다 라고 가정하는 것 입니다.

> "You shall know a word by the company it keeps" - John Firth

### Word2Vec을 구현하는 방법으로 Skip-Gram을 살펴보겠습니다

<img src="http://mccormickml.com/assets/word2vec/skip_gram_net_arch.png">

위 그림은 Skip-Gram 신경망 모식도 입니다.([Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/))

입력으로 단어벡터가 들어오고 출력부분에서는 입력단어와 다른 모든 단어들이 주변단어일 확률값을 학습하게 됩니다.

Skip-Gram 모델은 문장에서 가운데에 위치한 타겟단어를 입력받아 주변 단어를 예측하는 과정에서 학습이 됩니다.
예를 들어서 **"The tortoise jumped into the lake"** 라는 문장이 있고 윈도우 크기가 2인 경우 다음과 같이 skip-Gram을 학습하기 위한 데이터 쌍을 구축할 수 있습니다.

* 타겟: **The**, 주변 문맥단어: tortoise, jumped
    * 학습 샘플: (the, tortoise), (the, jumped)


* 타겟: **tortoise**, 주변 문맥단어: tortoise, jumped
    * 학습 샘플: (tortoise, the), (tortoise, jumped), (tortoise, into)


* 타겟: **jumped**, 주변 문맥단어: tortoise, jumped
    * 학습 샘플: (jumped, the), (jumped, tortoise), (jumped, into), (jumped, the)


* 타겟: **into**, 주변 문맥단어: tortoise, jumped
    * 학습 샘플: (into, tortoise), (into, jumped), (into, the), (into, lake)

                    ...
                    

이와같은 방법으로 전테 코퍼스에서 단어별로 슬라이딩하여 학습데이터를 생성하고 신경망을 학습합니다. 학습과정에서 효율을 높이기 위해 사용하는 기법들이 있지만 지금은 아직 신경망을 배우기 전이기 때문에 너무 깊게 들어가지 않겠습니다.

결과적으로 skip-gram 모델을 통해 결과로 단어 임베딩 벡터들을 얻게되어 단어, 문장들 간의 관련도 계산, 문서 분류같은 작업에 사용될 수 있습니다.

CountVectorizer, TF-IDF을 사용할 때 문맥정보를 보기위해 할 수 있는 최선의 방법은 bi-gram, tri-grams 같은 n-gram을 사용하는 것이었습니다. 하지만 skip-grams은 그 이상을 넘어 보다 강력한 문맥 정보를 제공해 줄 수 있습니다.



![alt text](https://www.researchgate.net/publication/304163783/figure/fig11/AS:668313973698561@1536349876279/Neural-Network-Architecture-for-CBoW-and-Skip-Gram-Model.ppm)

#### CBOW(Continuous Bag of Words)

Word2Vec을 구현하는 방법 중 CBOW는 skip-gram과 반대로 주변에 있는 문맥단어들을 가지고 타겟 단어 하나를 맞추는 과정을 통해 학습이 이루어 집니다.
예를 들어 입력이 (jumped, the, lake) 일 때 타겟 단어로 'into' 를 예측하며 학습을 합니다.  

하지만 Skip-gram이 같은 코퍼스를 사용했을 때 더 많은 학습 데이터를 만들어 낼 수 있기 때문에 임베딩 결과의 품질이 CBOW 보다 좋은 것으로 알려져 있습니다.



#### 그럼 임베딩 모델 학습은 어떻게 해야 할까요?
Word2Vec을 학습시키기 위해서는 충분히 큰 코퍼스를 학습시켜야 한다는 것을 알 수 있습니다. 다행인 것은 이미 충분히 큰 코퍼스들로 학습된 단어 임베딩을 쉽게 찾아 사용할 수 있다는 것입니다. 여러분이 지금까지 사용해온 Spacy 라이브러리의 모델도 Word2Vec 와 유사한 방식으로 학습한 임베딩을 제공합니다. 무지막지하게 큰 [Common Crawl](https://en.wikipedia.org/wiki/Common_Crawl) 데이터를 학습에 사용해 만든 모델이기 때문에 영어의 경우 충분히 대표성을 가진 임베딩이 나올 수 있을 것이라 생각할 수 있겠습니다.

이제 spacy를 통해 임베딩을 어떻게 사용하는지 살펴 봅시다.

In [ ]:
tokens = nlp("Dogs and cats school gagasdf")

# vector_norm: 벡터의 크기
for token in tokens:
    print(token.text, token.has_vector, token.vector_norm)

입력한 문장에 대한 임베딩 벡터를 얻습니다.

In [ ]:
vects = tokens.vector
print(vects)

In [ ]:
# 벡터의 차원을 보겠습니다
len(vects)

In [ ]:
# 두 문서를 만들어 코사인 유사도를 측정해 보겠습니다
doc1 = nlp("I found a wonderful restaurant")
doc2 = nlp("the food is delicious")

similarity = doc1.similarity(doc2)
print(similarity)

In [ ]:
doc3 = nlp("The restaurant we found yesterday is wonderful")

print(doc1.similarity(doc3))

In [ ]:
car = nlp('car')
bus = nlp('bus')
human = nlp('human')
monkey = nlp('monkey')
lion = nlp('lion')
gorilla = nlp('gorilla')
avengers = nlp('avengers')
marvel = nlp('marvel')

print('car vs bus : ', car.similarity(bus))
print('bus vs human : ', bus.similarity(human))
print('human vs monkey : ', human.similarity(monkey))
print('human vs lion : ', human.similarity(lion))
print('monkey vs gorilla : ', monkey.similarity(gorilla))
print('avengers vs marvel : ', avengers.similarity(marvel))

### PCA를 사용한 벡터 시각화
300 차원인 벡터들은 시각화 하기 어렵기 때문에 PCA를 사용해 2차원으로 변환해 보겠습니다.

In [ ]:
from sklearn.decomposition import PCA

def get_word_vectors(words):
    # 단어 벡터로 변환합니다
    return [nlp(word).vector for word in words]

words = ['car', 'truck', 'suv', 'bus', 'human', 'man', 'woman', 'monkey', 'fish' , 'shark', 'lion', 'tiger', 'avengers', 'marvel', 'thor', 'comics', 'superhero']

# PCA 모델의 차원을 설정하여 
pca = PCA(n_components=2)

# Fit & Transform
word_vect_2d = pca.fit_transform(get_word_vectors(words))

# 각 벡터가 300 차원에서 2차원으로 줄어 든 것을 확인 할 수 있습니다
word_vect_2d

In [ ]:
# 결과가 잘 보이도록 크기를 설정합니다
plt.figure(figsize=(15,10))

# 단어벡터를 그립니다
plt.scatter(word_vecs_2d[:,0], word_vecs_2d[:,1])

# 점 옆에 단어를 표시합니다
for word, coord in zip(words, word_vecs_2d):
    x, y = coord
    plt.text(x, y, word, size= 15)

plt.show()

In [ ]:
# 벡터 연산을 통해 단어간의 관계를 추론할 수 있습니다

from scipy.spatial.distance import cosine

king = nlp("king").vector
queen = nlp("queen").vector
man = nlp("man").vector
woman = nlp("woman").vector

# 단어벡터가 의미를 가진다면 다음과 같은 연산을 통해 result는 queen과 비슷한 뜻이 되겠지요?
result = king - man + woman

print("similarity between queen and (king - man + woman) : ", 1 - cosine(queen, result))

### 문서에서 벡터화 하여 KNN으로 검색해 보겠습니다.

Spacy를 사용해 문서를 임베딩 하겠습니다.

In [ ]:
X = [nlp(str(d)).vector for d in data]

In [ ]:
pd.DataFrame(X).shape

단어 queen 과 가장 유사한 단어를 찾아 보겠습니다.

In [ ]:
import numpy as np
most_similar= nlp.vocab.vectors.most_similar(np.array([queen]), n=10)

In [ ]:
most_similar[0][0]

In [ ]:
for key in most_similar[0][0]:
    print(nlp.vocab[key].text,)

문서 임베딩 벡터로 NN 모델을 학습합니다.

In [ ]:
nn_spacy = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn_spacy.fit(X)

Tfidf 벡터로 찾은 0번째 문서와 가장 유사한 문서 5개를 보겠습니다.

In [ ]:
nn.kneighbors([dtm.iloc[0].values])

Spacy 임베딩 모델로 찾아 봅시다.

In [ ]:
nn_spacy.kneighbors([X[0]])

문서62는 문서0과 동일하고 그 외, 문서 92를 가장 가깝다고 합니다.

In [ ]:
print(data[0][:150],'\n')
print(data[92][:150])

In [ ]:
print(data[83][:150])

In [ ]:
print(data[297][:150])

지금까지 자연어를 벡터로 표현하는 방법들에 대해 살펴보았습니다. Bag-of-Words 모델 중 단어의 출현 빈도를 사용해 텍스트 문서를 벡터로 변환하는 CounterVectorizer를 사용해 보았고, 문서별 단어의 빈도를 계산해 가중치를 적용한 TfidfVectorizer를 사용해 보았습니다.

이렇게 변환된 벡터는 코사인 유사도와 같은 방법을 통해 문서들 간 유사성을 수치로 나타낼 수 있었습니다. 문서들이 많을 때 코사인 유사도를 다 적용해서 가장 가까운 문서를 찾는 방법은 효율적이지 않습니다. 그래서 K-NN과 같은 트리 기반 알고리즘을 사용해 가장 가까운 K개의 문서를 빠르게 검색할 수 있었습니다.

BoW는 단어의 존재와 빈도를 중요시 여기는 대신 단어들의 순서정보를 무시하여 주변 문맥 정보가 없어지는 단점이 있었습니다. Word2Vec과 같은 담어 임베딩 방법은 벡터 생성 과정 중에 문맥 정보를 보존하여 유사한 의미를 가진 단어나 문장은 는 유사도가 큰 벡터가 됩니다.

## 참고자료

* Spacy - https://spacy.io/api
* Spacy 101 - https://course.spacy.io
* NLTK Book - https://www.nltk.org/book/
* An Introduction to Information Retrieval - https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf
* [A step by step explanation of PCA(principal component analysis)](http://localhost:8888/?token=ebeea486e072a985fe9597e449bd0f9cecbaec6ae8648532)